In [1]:
# data imports
import pickle
with open("corpora/corpus_chapters", "rb") as f:
    corpus_chapters = pickle.load(f)
with open("corpora/corpus_chapters_lemmas", "rb") as f:
    corpus_chapters_lemmas = pickle.load(f)
with open("corpora/corpus_chapters_lemmas_reduced", "rb") as f:
    corpus_chapters_lemmas_reduced = pickle.load(f)

In [2]:
rebuild = False

In [3]:
# prepare gensim model
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

if rebuild == True:
    corpus_chapters_dictionary = corpora.Dictionary(corpus_chapters)
    corpus_chapters_lemmas_dictionary = corpora.Dictionary(corpus_chapters_lemmas)
    corpus_chapters_lemmas_bow =  [corpus_chapters_lemmas_dictionary.doc2bow(chapter) 
                          for chapter in corpus_chapters_lemmas]

    # Set training parameters.
    num_topics = range(10,100,10)
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    corpus_chapters_lemmas_dictionary.filter_extremes(no_below=5, no_above=0.5)
    corpus_chapters_lemmas_bow =  [corpus_chapters_lemmas_dictionary.doc2bow(chapter) 
                          for chapter in corpus_chapters_lemmas]

    corpus_chapters_lemmas_reduced_dictionary = corpora.Dictionary(corpus_chapters_lemmas_reduced)
    corpus_chapters_lemmas_reduced_bow =  [corpus_chapters_lemmas_reduced_dictionary.doc2bow(chapter) 
                          for chapter in corpus_chapters_lemmas_reduced]
    #Doc2Vec
    corpus_chapters_lemmas_tagged = []
    for i, chapter in enumerate(corpus_chapters_lemmas):
        corpus_chapters_lemmas_tagged.append(TaggedDocument(chapter,[i]))
    doc2vec_model = Doc2Vec()
    doc2vec_model.build_vocab(corpus_chapters_lemmas_tagged)
    doc2vec_model.train(corpus_chapters_lemmas_tagged, total_examples=doc2vec_model.corpus_count, epochs=80)
    #pprint.pprint(dir(doc2vec_model))
    #for vec in doc2vec_model.docvecs:
    #    print(vec)
    #Similarity()
    doc2vec_model.save("models/doc2vec_model.model")

In [4]:
if rebuild == False:
    doc2vec_model = Doc2Vec.load("models/doc2vec_model.model")

In [5]:
import numpy as np
import pandas as pd
doc2vec_sim_dict = {}
for doc1 in range(0,2862):
    doc2vec_sim_dict[doc1] = []
    for doc2 in range(0,2862):
        #Iprint(doc1,doc2,doc2vec_model.dv.similarity(doc1,doc2))
        doc2vec_sim_dict[doc1].append(doc2vec_model.dv.similarity(doc1,doc2))

In [8]:
if rebuild == True:
    doc2vec_sim_df = pd.DataFrame.from_dict(doc2vec_sim_dict)
    doc2vec_sim_df.head()
    doc2vec_sim_df.to_pickle("doc2vec_sim_df.df")
else:
    doc2vec_sim_df = pd.read_pickle("models/doc2vec_sim_df.df")

In [9]:
#DOTAD UPORZADKOWANE

In [10]:
#top similar
print(doc2vec_sim_df[1].nlargest(11)[1:11])

39      0.489837
2596    0.441823
44      0.429102
178     0.426119
1192    0.419025
251     0.385512
611     0.367943
1252    0.365028
762     0.355356
2256    0.354491
Name: 1, dtype: float32


In [11]:
most_similar_docs

NameError: name 'most_similar_docs' is not defined

In [12]:
most_similar_docs = {}
for i in doc2vec_sim_df.index:
    if i not in most_similar_docs.keys():
        most_similar_docs[i] = {}
    largest_i = list(doc2vec_sim_df[i].nlargest(11).index)
    #print(largest_i)
    #most_similar_docs[i] = dict.fromkeys(largest_i, {"year":'', "head":'',"text":''})
    #print(most_similar_docs[i])
    year = ''
    head = ''
    text = ''
    for n_i in largest_i:
        #print(n_i)        
        try:
            year = search_text_chapter(n_i, "year")
            
        except Exception:
            pass
        
        try:
            head = search_text_chapter(n_i, "head")
        except Exception:
            pass
        
        try:
            text = search_text_chapter(n_i, "text")
        except Exception:
            pass
        
        #print(year)
        if n_i not in most_similar_docs[i].keys():
            most_similar_docs[i][n_i]={"year":None,"head":None,"text":None}
            most_similar_docs[i][n_i]["year"] = year
            most_similar_docs[i][n_i]["head"] = head
            most_similar_docs[i][n_i]["text"] = text

In [13]:
most_similar_docs[1110]

{1110: {'year': '', 'head': '', 'text': ''},
 1098: {'year': '', 'head': '', 'text': ''},
 2349: {'year': '', 'head': '', 'text': ''},
 1106: {'year': '', 'head': '', 'text': ''},
 956: {'year': '', 'head': '', 'text': ''},
 1498: {'year': '', 'head': '', 'text': ''},
 2309: {'year': '', 'head': '', 'text': ''},
 1463: {'year': '', 'head': '', 'text': ''},
 2505: {'year': '', 'head': '', 'text': ''},
 2525: {'year': '', 'head': '', 'text': ''},
 1520: {'year': '', 'head': '', 'text': ''}}

In [15]:
docs_by_year = [] 
docs_by_liber = []
docs_by_chapter = [] # list of chapters (liber, chapter, year, tokens)
#for liber in [2]:
tmp = []
for liber in text_dict.keys():
    for chapter in text_dict[liber].keys():
        chapter_tokens = []
        for txt_arr in text_dict[liber][chapter]:
            #txt_arr_toks = []
            if txt_arr[1] == "text": #exclude head etc.   
                #print(txt_arr)
                for tok in txt_arr[2]: #iterates over tokens                    
                    chapter_tokens.append((liber, chapter, txt_arr[0], tok[0], tok[1], tok[2])) #liber,chap, year,token
            else:
                tmp.append(txt_arr)            
                
        if len(chapter_tokens) > 0:
            docs_by_chapter.append(chapter_tokens)

NameError: name 'text_dict' is not defined

In [14]:
search_text_chapter(1,'year')

NameError: name 'search_text_chapter' is not defined

In [16]:
#search_text_chapter(2,"head")
text_to_all_chapters[2]
text_dict_joined[0][2]

NameError: name 'text_to_all_chapters' is not defined

In [17]:
list(doc2vec_sim_df[0].nlargest(11).index)

[0, 1029, 236, 2381, 461, 123, 1330, 595, 174, 730, 2026]

In [18]:
doc2vec_sim_df[0].nlargest(11)

0       1.000000
1029    0.562609
236     0.556832
2381    0.554758
461     0.546207
123     0.541571
1330    0.541421
595     0.535762
174     0.526044
730     0.516737
2026    0.507058
Name: 0, dtype: float32

In [ ]:
# Mrebuildn index tonlargestctionary.
temp = corpus_chapters_lemmas_reduced_dictionary[0]  # This is only to "load" the dictionary.
id2word = corpus_chapters_lemmas_reduced_dictionary.id2token

lda_model_lemmas_reduced = LdaModel(
    corpus=corpus_chapters_lemmas_reduced_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=20,
    passes=passes,
    eval_every=eval_every
)

In [19]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_lemmas_reduced, corpus_chapters_lemmas_reduced_bow, corpus_chapters_lemmas_reduced_dictionary)
vis

ModuleNotFoundError: No module named 'pyLDAvis'

In [20]:
import pandas as pd
chapters_topic_dists ={}
for i, chapter in enumerate(lda_model_lemmas_reduced.get_document_topics(corpus_chapters_lemmas_reduced_bow)):
    chapters_topic_dists[i] = dict.fromkeys(range(0,20),0)
    for topic, prob in chapter:        
        chapters_topic_dists[i][topic] = prob
#print(chapters_topic_dists)
chapters_topic_matrix = pd.DataFrame.from_dict(chapters_topic_dists,orient="index")
chapters_topic_matrix.tail()

NameError: name 'lda_model_lemmas_reduced' is not defined

In [21]:
import matplotlib
chapters_topic_matrix.plot()

NameError: name 'chapters_topic_matrix' is not defined

In [ ]:
import plotly
import plotly.express as px
chapters_topic_matrix["doc"] = chapters_topic_matrix.index
chapters_topic_matrix_years = { doc:year for doc, year in enumerate(corpus_chapters_lemmas_year) }
chapters_topic_matrix["year"] = corpus_chapters_lemmas_year
chapters_topic_matrix.head()
chapters_topic_matrix_wide = chapters_topic_matrix.melt(id_vars=["doc","year"],var_name="topic")

In [ ]:
len(corpus_chapters_lemmas_year)

In [ ]:
chapters_topic_matrix_wide = chapters_topic_matrix_wide[chapters_topic_matrix_wide["value"] > 0].sort_values(["doc","topic"])
chapters_topic_matrix_wide.head()

In [ ]:
#px.scatter_matrix(chapters_topic_matrix, x="doc", y="value", color=topic)
import plotly
import plotly.express as px
fig = px.scatter(chapters_topic_matrix_wide[chapters_topic_matrix_wide["value"] >0.5], x="doc", y="value",
                 color="topic",
                 #size="value",
                 facet_col="topic", facet_col_wrap=2, height=2000)
#fig.update_layout(yaxis_range=[0.2,1.2])
fig.show()

In [ ]:
#px.scatter_matrix(chapters_topic_matrix, x="doc", y="value", color=topic)
import plotly
import plotly.express as px
fig = px.scatter(chapters_topic_matrix_wide[chapters_topic_matrix_wide["value"] >0.3], x="year",
                 y="value",
                 color="topic",
                 #size="value",
                 facet_col="topic", facet_col_wrap=2, height=2000)
#fig.update_layout(yaxis_range=[0.2,1.2])
fig.update_xaxes(categoryorder='array', categoryarray= corpus_chapters_lemmas_year)
fig.show()

In [ ]:
import numpy as np
import pandas as pd
topic_sims = []

for similarities in index:
    topic_sims.append(similarities)

In [ ]:
chapters_topic_sim = pd.DataFrame(topic_sims)
chapters_topic_sim.head()

In [ ]:
for row in topic_matrix:
    print(topic_matrix[row] > 0.5)

In [ ]:
for num_topic in num_topics:
    lda_model_lemmas_rednlargested = LdaModel(
        corpus=corpus_chapters_lemmas_reduced_bow,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topic,
        passes=passes,
        eval_every=eval_every
    )
    coherence_model_lda = CoherenceModel(model=lda_model_lemmas_reduced, texts=corpus_chapters_lemmas_reduced, 
                                         dictionary=corpus_chapters_lemmas_reduced_dictionary,
                                         coherence='c_v')
    
    print("topics: " + str(num_topic) ,"coherence: ", coherence_model_lda.get_coherence(), 
          "\tperplexity: ", 
          lda_model_lemmas_reduced.log_perplexity(corpus_chapters_lemmas_reduced_bow)
         )
#best results
#topics: 10 coherence:  0.4043357272535478 	perplexity:  -7.509568616109219
#topics: 20 coherence:  0.45175391591224745 	perplexity:  -7.559552059787899
#topics: 30 coherence:  0.4145315890018234 	perplexity:  -7.603553920253678
#topics: 40 coherence:  0.41797913503984957 	perplexity:  -7.663085551756763
#topics: 50 coherence:  0.3995843728274568 	perplexity:  -7.728750670838585

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_lemmas_reduced, texts=corpus_chapters_lemmas_reduced, 
                                     dictionary=corpus_chapters_lemmas_reduced_dictionary,
                                     coherence='c_v')

print("topics: " + str(20) ,"coherence: ", coherence_model_lda.get_coherence(), 
      "\tperplexity: ", 
      lda_model_lemmas_reduced.log_perplexity(corpus_chapters_lemmas_reduced_bow)
     )

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_lemmas_reduced, corpus_chapters_lemmas_reduced_bow, corpus_chapters_lemmas_reduced_dictionary)
vis

In [ ]:
corpus_chapters_lemmas_dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus_chapters_lemmas_bow =  [corpus_chapters_lemmas_dictionary.doc2bow(chapter) 
                      for chapter in corpus_chapters_lemmas
                              ]

# Make an index to word dictionary.
temp = corpus_chapters_lemmas_dictionary[0]  # This is only to "load" the dictionary.
id2word = corpus_chapters_lemmas_dictionary.id2token

for num_topic in num_topics:
    lda_model_lemmas = LdaModel(
        corpus=corpus_chapters_lemmas_bow,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topic,
        passes=passes,
        eval_every=eval_every
    )
    coherence_model_lda = CoherenceModel(model=lda_model_lemmas, texts=corpus_chapters_lemmas, 
                                         dictionary=corpus_chapters_lemmas_dictionary,
                                         coherence='c_v')
    
    print("topics: " + str(num_topic) ,"coherence: ", coherence_model_lda.get_coherence(), "\tperplexity: ", 
          lda_model_lemmas.log_perplexity(corpus_chapters_lemmas_bow))

In [ ]:
import re
from nltk.probability import ConditionalFreqDist
word_len_cfd = ConditionalFreqDist()
chapter_len_cfd = ConditionalFreqDist()
year_len_cfd = ConditionalFreqDist()
liber_len_cfd = ConditionalFreqDist()
chapters_nltk_categorized.fileids()
chapters_word_length = {"by_year" : {}, "by_chapter" : {}, "by_liber" : {}}
for file in chapters_nltk_categorized.fileids():
    file_id = re.sub(r'\.txt', '', file)
    #print(file_id)
    #print(search_text_chapter(int(file_id), "year"))
    year = search_text_chapter(int(file_id), "year")
    chapter = search_text_chapter(int(file_id), "chapter")
    liber = search_text_chapter(int(file_id), "liber")
    #word_lens = []
    for w in chapters_nltk_categorized.words(file):
        if w not in [';', ',', '.', '-', '"', '?', '!']:
            word_len_cfd[len(w)][w] +=1
            chapter_len_cfd[chapter][len(w)] +=1
            year_len_cfd[year][len(w)] +=1
            liber_len_cfd[liber][len(w)] +=1

In [ ]:
# topic modelling
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)